## PREPARE | Collect Data

The data used for this analysis is a public domain (CC0 1.0 Universal Public Domain) dataset made available on Kaggle (via user MÖBIUS).

Data source: [FitBit Fitness Tracker Data](https://www.kaggle.com/datasets/arashnic/fitbit)  

### Data Details
The data uploaded by Kaggle user MÖBIUS originates from respondents to a distributed survey via Amazon Mechanical Turk from March 12 to May 12, 2016. Thirty-three Fitbit users submitted personal tracker data collected in 18 files covering:
- Physical Activity (measured in Steps, Calories, and METs (metabolic equivalents)
- Sleep (measured in minutes)
- Heart rate (bpm)
- Weight/BMI (lbs/kg)

Data covers a 30-day period (04.12.16 - 05.12.16).



This analysis will focus on **Physical Activity** (daily, hourly), **Sleep** (daily), and **weight/bmi** to understand usage at a broader level. 


While 33 unique individuals provided data for physical activity, the other measured data contained fewer individuals.  
- Physical Activity: 33
- Sleep monitoring: 24
- Weight: 8
- Heart rate: 14 

### Licensing, Privacy, Security, Integrity
[CC0 1.0 Universal Public Domain](https://creativecommons.org/publicdomain/zero/1.0/)

All users participating in the survey consented to the submission of personal tracking data. The users' privacy has been protected by only identifying unique individuals via randomly generated ID numbers. This data has been provided by a 3rd party, the Kaggle user MÖBIUS.

### Data Integrity
- Sample selection bias since, insignificant 
- Variation in output due to different types of Fitbit trackers
- Variation in individual tracking behavior/preferences
- Concerns that there is no demographic data (sex, age, location) 
- Obsolescence - 5 years old



In [17]:
import pandas as pd
import pandas_gbq
import os

from google.cloud import bigquery
%load_ext google.cloud.bigquery

pandas_gbq.context.project = 'gac-bellabeat'
pandas_gbq.context.dialect = 'standard'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/Users/atat/code/phlln/gcp_keys/gac-bellabeat-jupyter-bigquery-key.json'

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


In [19]:
%%bigquery
SELECT *
  FROM `gac-bellabeat.activity.sleepdaily`
  LIMIT 1

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.62s/rows]


,Id,SleepDay,TotalSleepRecords,TotalMinutesAsleep,TotalTimeInBed
0,1503960366,2016-04-12 00:00:00+00:00,1,327,346


## PROCESS | Wrangle Data

Inspect, Wrangle, Validate Data
- Check data type, number of records, number of nulls, stat summary
- addition of columns for easier analysis _by day of week and hour
- We'll look at Physical Activity, Sleep, Weight/BMI

### Data Details
The data uploaded by Kaggle user MÖBIUS originates from respondents to a distributed survey via Amazon Mechanical Turk from March 12 to May 12, 2016. Thirty-three Fitbit users submitted personal tracker data collected in 18 files covering:
- Physical Activity (measured in Steps, Calories, and METs (metabolic equivalents)
- Sleep (measured in minutes)
- Heart rate (bpm)
- Weight/BMI (lbs/kg)

Data covers a 30-day period (04.12.16 - 05.12.16).

This analysis will focus on **Physical Activity** (daily, hourly), **Sleep** (daily), and **weight/bmi** to understand usage at a broader level. 


### Activity Data

The most detailed and complete data is for logging physical activity. Various metrics for measuring physical exertion (Steps, Calories, METs) are provided whether measure by time or distance. Additionally, levels of intensity have been categorized into four levels ('VeryActive', 'FairlyActive', 'LightlyActive', 'Sedentary'). This data has also been recorded at differnt time scales (daily, hourly, and minute).   

#### Daily Activity Data

##### Validate number of unique individuals

In [81]:
%%bigquery
SELECT
    COUNT(DISTINCT ID) AS num_unique_users
FROM `gac-bellabeat.activity.daily`;
  

Downloading: 100%|██████████| 1/1 [00:02<00:00,  2.04s/rows]


,num_unique_users
0,33


##### Check for Null Values

In [87]:
%%bigquery
SELECT
    COUNT(*) total_rows,
    SUM(CASE WHEN ActivityDate IS NULL THEN 1 ELSE 0 END) activitydate_num_null,
    SUM(CASE WHEN TotalSteps IS NULL THEN 1 ELSE 0 END) totalsteps_num_null,
    SUM(CASE WHEN VeryActiveMinutes IS NULL THEN 1 ELSE 0 END) veryactive_num_null,
    SUM(CASE WHEN FairlyActiveMinutes IS NULL THEN 1 ELSE 0 END) fairlyactive_num_null,
    SUM(CASE WHEN LightlyActiveMinutes IS NULL THEN 1 ELSE 0 END) lightlyactive_num_null,
    SUM(CASE WHEN SedentaryMinutes IS NULL THEN 1 ELSE 0 END) sedentary_num_null,
    SUM(CASE WHEN Calories IS NULL THEN 1 ELSE 0 END) calories_num_null
FROM `gac-bellabeat.activity.daily`;

Downloading: 100%|██████████| 1/1 [00:01<00:00,  1.42s/rows]


,total_rows,activitydate_num_null,totalsteps_num_null,veryactive_num_null,fairlyactive_num_null,lightlyactive_num_null,sedentary_num_null,calories_num_null
0,940,0,0,0,0,0,0,0


##### Create summary stats panel (Total, Mean, Min, 25%, 50%, 75%, Max, Std)

In [91]:
%%bigquery
SELECT 1 AS Index, 
      'Count' AS Statistic,
       SUM(TotalSteps) AS total_steps,
       SUM(VeryActiveMinutes) AS very_active_minutes,
       SUM(FairlyActiveMinutes) AS fairly_active_minutes,
       SUM(LightlyActiveMinutes) AS lightly_active_minutes,
       SUM(SedentaryMinutes) AS sedentary_minutes
  FROM `gac-bellabeat.activity.daily`
 UNION ALL
SELECT 2, 
      'Mean',
       ROUND(AVG(TotalSteps), 1) AS mean_total_steps,
       ROUND(AVG(VeryActiveMinutes), 1) AS mean_veryactive_minutes,
       ROUND(AVG(FairlyActiveMinutes), 1) AS mean_fairlyactive_minutes,
       ROUND(AVG(LightlyActiveMinutes), 1) AS mean_lightlyactive_minutes,
       ROUND(AVG(SedentaryMinutes), 1) AS mean_sedentary_minutes
  FROM `gac-bellabeat.activity.daily`
  UNION ALL
SELECT 3,
       'Min',
       MIN(TotalSteps) AS min_total_steps,
       MIN(VeryActiveMinutes) AS min_veryactive_minutes,
       MIN(FairlyActiveMinutes) AS min_fairlyactive_minutes,
       MIN(LightlyActiveMinutes) AS min_lightlyactive_minutes,
       MIN(SedentaryMinutes) AS min_sedentary_minutes
  FROM `gac-bellabeat.activity.daily`
  UNION ALL
(SELECT 4,
       '25%',
       PERCENTILE_CONT(TotalSteps, 0.25) OVER() AS percentile25_total_steps,
       PERCENTILE_CONT(VeryActiveMinutes, 0.25) OVER() AS percentile25_veryactive_minutes,
       PERCENTILE_CONT(FairlyActiveMinutes, 0.25) OVER() AS percentile25_fairlyactive_minutes,
       PERCENTILE_CONT(LightlyActiveMinutes, 0.25) OVER() AS percentile25_lightlyactive_minutes,
       PERCENTILE_CONT(SedentaryMinutes, 0.25) OVER() AS percentile25_sedentary_minutes
  FROM `gac-bellabeat.activity.daily`
  LIMIT 1) 
  UNION ALL 
(SELECT 5,
       '50%',
       PERCENTILE_CONT(TotalSteps, 0.50) OVER() AS percentile50_total_steps,
       PERCENTILE_CONT(VeryActiveMinutes, 0.50) OVER() AS percentile50_veryactive_minutes,
       PERCENTILE_CONT(FairlyActiveMinutes, 0.50) OVER() AS percentile50_fairlyactive_minutes,
       PERCENTILE_CONT(LightlyActiveMinutes, 0.50) OVER() AS percentile50_lightlyactive_minutes,
       PERCENTILE_CONT(SedentaryMinutes, 0.50) OVER() AS percentile50_sedentary_minutes
  FROM `gac-bellabeat.activity.daily`
  LIMIT 1) 
  UNION ALL 
(SELECT 6,
       '75%',
       PERCENTILE_CONT(TotalSteps, 0.75) OVER() AS percentile75_total_steps,
       PERCENTILE_CONT(VeryActiveMinutes, 0.75) OVER() AS percentile75_veryactive_minutes,
       PERCENTILE_CONT(FairlyActiveMinutes, 0.75) OVER() AS percentile75_fairlyactive_minutes,
       PERCENTILE_CONT(LightlyActiveMinutes, 0.75) OVER() AS percentile75_lightlyactive_minutes,
       PERCENTILE_CONT(SedentaryMinutes, 0.75) OVER() AS percentile75_sedentary_minutes
  FROM `gac-bellabeat.activity.daily`
  LIMIT 1) 
UNION ALL
SELECT 7,
       'Max',
       MAX(TotalSteps) AS max_total_steps,
       MAX(VeryActiveMinutes) AS max_veryactive_minutes,
       MAX(FairlyActiveMinutes) AS max_fairlyactive_minutes,
       MAX(LightlyActiveMinutes) AS max_lightlyactive_minutes,
       MAX(SedentaryMinutes) AS max_sedentary_minutes
  FROM `gac-bellabeat.activity.daily`
UNION ALL
(SELECT 8,
       'STD',
       ROUND(STDDEV_POP(TotalSteps) OVER(), 1) AS std_total_steps,
       ROUND(STDDEV_POP(VeryActiveMinutes) OVER(), 1) AS std_veryactive_minutes,
       ROUND(STDDEV_POP(FairlyActiveMinutes) OVER(), 1) AS std_fairlyactive_minutes,
       ROUND(STDDEV_POP(LightlyActiveMinutes) OVER(), 1) AS std_lightlyactive_minutes,
       ROUND(STDDEV_POP(SedentaryMinutes) OVER(), 1) AS std_sedentary_minutes
FROM `gac-bellabeat.activity.daily`
LIMIT 1)
ORDER BY Index ASC;

Downloading: 100%|██████████| 8/8 [00:01<00:00,  5.68rows/s]


,Index,Statistic,total_steps,very_active_minutes,fairly_active_minutes,lightly_active_minutes,sedentary_minutes
0,1,Count,7179636.00,19895.0,12751.0,181244.0,931738.00
1,2,Mean,7637.90,21.2,13.6,192.8,991.20
2,3,Min,0.00,0.0,0.0,0.0,0.00
3,4,25%,3789.75,0.0,0.0,127.0,729.75
4,5,50%,7405.50,4.0,6.0,199.0,1057.50
5,6,75%,10727.00,32.0,19.0,264.0,1229.50
6,7,Max,36019.00,210.0,143.0,518.0,1440.00
7,8,STD,5084.40,32.8,20.0,109.1,301.10


##### Create new DayOfWeek column extracted from ActivityDate to faciliate analysis

In [93]:
%%bigquery
SELECT Id,
       ActivityDate,
       FORMAT_DATE('%a', ActivityDate) AS DayOfWeek,
       TotalSteps,
       VeryActiveMinutes,
       FairlyActiveMinutes,
       LightlyActiveMinutes,
       SedentaryMinutes, 
       Calories
FROM `gac-bellabeat.activity.daily`;

Downloading: 100%|██████████| 940/940 [00:01<00:00, 773.58rows/s]


,Id,ActivityDate,DayOfWeek,TotalSteps,VeryActiveMinutes,FairlyActiveMinutes,LightlyActiveMinutes,SedentaryMinutes,Calories
0,1624580081,2016-05-01,Sun,36019,186,63,171,1020,2690
1,1644430081,2016-04-14,Thu,11037,5,58,252,1125,3226
2,1644430081,2016-04-19,Tue,11256,5,58,278,1099,3300
3,1644430081,2016-04-28,Thu,9405,3,53,227,1157,3108
4,1644430081,2016-04-30,Sat,18213,9,71,402,816,3846
...,...,...,...,...,...,...,...,...,...
935,1844505072,2016-04-20,Wed,8,0,0,1,1439,1349
936,4020332650,2016-04-17,Sun,16,0,0,2,1438,1990
937,4319703577,2016-05-12,Thu,17,0,0,2,0,257
938,6775888955,2016-05-03,Tue,9,0,0,1,1439,1843


### Hourly Activity Data

There are three tables (hourly_steps, hourly_calories, hourly_intensities) recording hourly data. Let's check if there are any differences among the three tables prior to merging them on the columns Id and ActivityHour as a compound key. We'll do that by comparing a pair of tables at a time and using EXCEPT, reverse the positions of the tables, and then UNION ALL the result. If there are any combinations of Id, Activity that are only present in one of the tables, we should see that combination in our query results.

In [98]:
%%bigquery
(SELECT hs.Id, hs.ActivityHour
   FROM `gac-bellabeat.activity.hourly_steps` hs
 EXCEPT DISTINCT
 SELECT hc.Id, hc.ActivityHour
  FROM `gac-bellabeat.activity.hourly_calories` hc)
UNION ALL
(SELECT hc.Id, hc.ActivityHour
   FROM `gac-bellabeat.activity.hourly_calories` hc
 EXCEPT DISTINCT
 SELECT hs.Id, hs.ActivityHour
  FROM `gac-bellabeat.activity.hourly_steps` hs);

Query complete after 0.02s: 100%|██████████| 7/7 [00:00<00:00, 369.36query/s]                         
Downloading: 0rows [00:00, ?rows/s]


,Id,ActivityHour


In [99]:
%%bigquery
(SELECT hi.Id, hi.ActivityHour
   FROM `gac-bellabeat.activity.hourly_intensities` hi
 EXCEPT DISTINCT
 SELECT hc.Id, hc.ActivityHour
   FROM `gac-bellabeat.activity.hourly_calories` hc)
UNION ALL
(SELECT hc.Id, hc.ActivityHour
   FROM `gac-bellabeat.activity.hourly_calories` hc
 EXCEPT DISTINCT
 SELECT hi.Id, hi.ActivityHour
   FROM `gac-bellabeat.activity.hourly_intensities` hi);

Query complete after 0.01s: 100%|██████████| 7/7 [00:00<00:00, 827.93query/s]                         
Downloading: 0rows [00:00, ?rows/s]


,Id,ActivityHour


Since there are no resulting rows from our queries, we can safely merge all three tables on the compound keys Id, ActivityHour.

In [107]:
%%bigquery
SELECT *
  FROM `gac-bellabeat.activity.hourly_intensities` hi
  JOIN `gac-bellabeat.activity.hourly_steps` hs
    ON hi.Id = hs.Id
   AND hi.ActivityHour = hs.ActivityHour
  JOIN  `gac-bellabeat.activity.hourly_calories` hc
    ON hi.Id = hc.Id
   AND hi.ActivityHour = hc.ActivityHour  
 ORDER BY hi.Id, hi.ActivityHour

Downloading: 100%|██████████| 22099/22099 [00:01<00:00, 12702.06rows/s]


,Id,ActivityHour,TotalIntensity,AverageIntensity,Id_1,ActivityHour_1,StepTotal,Id_2,ActivityHour_2,Calories
0,1503960366,2016-04-12 00:00:00+00:00,20,0.333333,1503960366,2016-04-12 00:00:00+00:00,373,1503960366,2016-04-12 00:00:00+00:00,81
1,1503960366,2016-04-12 01:00:00+00:00,8,0.133333,1503960366,2016-04-12 01:00:00+00:00,160,1503960366,2016-04-12 01:00:00+00:00,61
2,1503960366,2016-04-12 02:00:00+00:00,7,0.116667,1503960366,2016-04-12 02:00:00+00:00,151,1503960366,2016-04-12 02:00:00+00:00,59
3,1503960366,2016-04-12 03:00:00+00:00,0,0.000000,1503960366,2016-04-12 03:00:00+00:00,0,1503960366,2016-04-12 03:00:00+00:00,47
4,1503960366,2016-04-12 04:00:00+00:00,0,0.000000,1503960366,2016-04-12 04:00:00+00:00,0,1503960366,2016-04-12 04:00:00+00:00,48
...,...,...,...,...,...,...,...,...,...,...
22094,8877689391,2016-05-12 10:00:00+00:00,12,0.200000,8877689391,2016-05-12 10:00:00+00:00,514,8877689391,2016-05-12 10:00:00+00:00,126
22095,8877689391,2016-05-12 11:00:00+00:00,29,0.483333,8877689391,2016-05-12 11:00:00+00:00,1407,8877689391,2016-05-12 11:00:00+00:00,192
22096,8877689391,2016-05-12 12:00:00+00:00,93,1.550000,8877689391,2016-05-12 12:00:00+00:00,3135,8877689391,2016-05-12 12:00:00+00:00,321
22097,8877689391,2016-05-12 13:00:00+00:00,6,0.100000,8877689391,2016-05-12 13:00:00+00:00,307,8877689391,2016-05-12 13:00:00+00:00,101


To make take this merged table easier to work with for analysis, we'll remove the duplicate columns (Id, ActivityHour) and extract separate HourOfDay and DayOfWeek columns from ActivityHour, and then save it as a view (`gac-bellabeat.activity.hourly`) to access later in our analysis.

In [113]:
%%bigquery
SELECT hi.Id,
       hi.ActivityHour,
       EXTRACT(hour FROM hi.ActivityHour) AS HourOfDay,
       FORMAT_DATE('%a', hi.ActivityHour) AS DayOfWeek,
       TotalIntensity,
       AverageIntensity,
       StepTotal,
       Calories
  FROM `gac-bellabeat.activity.hourly_intensities` hi
  JOIN `gac-bellabeat.activity.hourly_steps` hs
    ON hi.Id = hs.Id
   AND hi.ActivityHour = hs.ActivityHour
  JOIN  `gac-bellabeat.activity.hourly_calories` hc
    ON hi.Id = hc.Id
   AND hi.ActivityHour = hc.ActivityHour  
 ORDER BY hi.Id, hi.ActivityHour
              

Downloading: 100%|██████████| 22099/22099 [00:01<00:00, 12811.76rows/s]


,Id,ActivityHour,HourOfDay,DayOfWeek,TotalIntensity,AverageIntensity,StepTotal,Calories
0,1503960366,2016-04-12 00:00:00+00:00,0,Tue,20,0.333333,373,81
1,1503960366,2016-04-12 01:00:00+00:00,1,Tue,8,0.133333,160,61
2,1503960366,2016-04-12 02:00:00+00:00,2,Tue,7,0.116667,151,59
3,1503960366,2016-04-12 03:00:00+00:00,3,Tue,0,0.000000,0,47
4,1503960366,2016-04-12 04:00:00+00:00,4,Tue,0,0.000000,0,48
...,...,...,...,...,...,...,...,...
22094,8877689391,2016-05-12 10:00:00+00:00,10,Thu,12,0.200000,514,126
22095,8877689391,2016-05-12 11:00:00+00:00,11,Thu,29,0.483333,1407,192
22096,8877689391,2016-05-12 12:00:00+00:00,12,Thu,93,1.550000,3135,321
22097,8877689391,2016-05-12 13:00:00+00:00,13,Thu,6,0.100000,307,101


Next, we'll save this table as a view to use later in our analysis.

Let's run our summary stat panel on this data. 

In [116]:
%%bigquery
SELECT 1 AS Index, 
      'Count' AS Statistic,
       SUM(TotalIntensity) AS total_intensity,
       SUM(AverageIntensity) AS avg_intensity,
       SUM(StepTotal) AS step_total,
       SUM(Calories) AS calories
  FROM `gac-bellabeat.activity.hourly`
 UNION ALL
SELECT 2, 
      'Mean',
       ROUND(AVG(TotalIntensity), 1) AS mean_total_intensity,
       ROUND(AVG(AverageIntensity), 1) AS mean_avg_intensity,
       ROUND(AVG(StepTotal), 1) AS mean_step_total,
       ROUND(AVG(Calories), 1) AS mean_calories
  FROM `gac-bellabeat.activity.hourly`
  UNION ALL
SELECT 3,
       'Min',
       MIN(TotalIntensity) AS min_total_steps,
       MIN(AverageIntensity) AS min_avg_intensity,
       MIN(StepTotal) AS min_step_total,
       MIN(Calories) AS min_calories
  FROM `gac-bellabeat.activity.hourly`
  UNION ALL
(SELECT 4,
       '25%',
       PERCENTILE_CONT(TotalIntensity, 0.25) OVER() AS percentile25_total_intensity,
       PERCENTILE_CONT(AverageIntensity, 0.25) OVER() AS percentile25_avg_intensitys,
       PERCENTILE_CONT(StepTotal, 0.25) OVER() AS percentile25_step_total,
       PERCENTILE_CONT(Calories, 0.25) OVER() AS percentile25_calories
  FROM `gac-bellabeat.activity.hourly`
  LIMIT 1) 
  UNION ALL 
(SELECT 5,
       '50%',
       PERCENTILE_CONT(TotalIntensity, 0.50) OVER() AS percentile50_total_intensity,
       PERCENTILE_CONT(AverageIntensity, 0.50) OVER() AS percentile50_avg_intensity,
       PERCENTILE_CONT(StepTotal, 0.50) OVER() AS percentile50_step_total,
       PERCENTILE_CONT(Calories, 0.50) OVER() AS percentile50_calories
  FROM `gac-bellabeat.activity.hourly`
  LIMIT 1) 
  UNION ALL 
(SELECT 6,
       '75%',
       PERCENTILE_CONT(TotalIntensity, 0.75) OVER() AS percentile75_total_intensity,
       PERCENTILE_CONT(AverageIntensity, 0.75) OVER() AS percentile75_avg_intensity,
       PERCENTILE_CONT(StepTotal, 0.75) OVER() AS percentile75_step_total,
       PERCENTILE_CONT(Calories, 0.75) OVER() AS percentile75_calories
  FROM `gac-bellabeat.activity.hourly`
  LIMIT 1) 
UNION ALL
SELECT 7,
       'Max',
       MAX(TotalIntensity) AS max_total_intensity,
       MAX(AverageIntensity) AS max_avg_intensity,
       MAX(StepTotal) AS max_step_total,
       MAX(Calories) AS max_calories
  FROM `gac-bellabeat.activity.hourly`
UNION ALL
(SELECT 8,
       'STD',
       ROUND(STDDEV_POP(TotalIntensity) OVER(), 1) AS std_total_intensity,
       ROUND(STDDEV_POP(AverageIntensity) OVER(), 1) AS std_avg_intensity,
       ROUND(STDDEV_POP(StepTotal) OVER(), 1) AS std_step_total,
       ROUND(STDDEV_POP(Calories) OVER(), 1) AS std_calories
FROM `gac-bellabeat.activity.hourly`
LIMIT 1)
ORDER BY Index ASC;

Downloading: 100%|██████████| 8/8 [00:01<00:00,  5.31rows/s]


,Index,Statistic,total_intensity,avg_intensity,step_total,calories
0,1,Count,265969.0,4432.816847,7075356.0,2152150.0
1,2,Mean,12.0,0.200000,320.2,97.4
2,3,Min,0.0,0.000000,0.0,42.0
3,4,25%,0.0,0.000000,0.0,63.0
4,5,50%,3.0,0.050000,40.0,83.0
5,6,75%,16.0,0.266667,357.0,108.0
6,7,Max,180.0,3.000000,10554.0,948.0
7,8,STD,21.1,0.400000,690.4,60.7


To make take this merged table easier to work with for analysis, we'll remove the duplicate columns (Id, ActivityHour) and create a DayOfWeek column.